# Experiments

Roughly speaking, I will try to use the models trained on the datasets, then remove the FC layer and add other classifiers. Then I will report the result of the other classifiers on the final performance using either EER or accuracy.

In [1]:
data_path = '../data/mmuisd/normal/pocket'
results_path = '../results/'

# data_path = '/content/drive/MyDrive/IJCNN/data/osaka'
# results_path = '/content/drive/MyDrive/IJCNN/results'

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp -r "/content/drive/MyDrive/IJCNN/src" /content/
!mv src/datasets.py datasets.py
!mv src/models.py models.py
!mv src/analyze.py analyze.py
!ls

analyze.py  datasets.py  drive	models.py  sample_data	src


In [2]:
import datasets, models, analyze
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# import nevergrad as ng
import os
np.set_printoptions(precision=4)
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [4]:
def prepare_data(train_sps, val_sps, test_sps, max_cls, split, data, labels, indices_list, run_n):
    train_split = int(split[0] * max_cls)
    val_split = int(split[2] * max_cls)

    rand_indices = np.random.choice(range(max_cls), size=max_cls, replace=False)
    indices_list.append(rand_indices)
    print('Run #%d:'%run_n)
    
    all_data_rand, all_labels_rand = [data[x] for x in rand_indices], [labels[x] for x in rand_indices]
    b_data, b_labels = datasets.generate_batch(train_sps, all_data_rand[:train_split], all_labels_rand[:train_split])
    
    val_data, val_labels = datasets.generate_batch(val_sps, all_data_rand[train_split:train_split+val_split], all_labels_rand[train_split:train_split+val_split])
    ridx = np.random.choice(range(b_data[0].shape[0]), size=b_data[0].shape[0], replace=False)
    
    b_data_test, b_labels_test = datasets.generate_batch(test_sps, all_data_rand[train_split+val_split:], all_labels_rand[train_split+val_split:])

    l_input = b_data[0][ridx]
    r_input = b_data[1][ridx]
    b_labels = b_labels[ridx]

    # print(l_input[0].shape)

    l_input_val = val_data[0]
    r_input_val = val_data[1]

    l_input_test = b_data_test[0]
    r_input_test = b_data_test[1]
    return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test

In [5]:
def load_dataset(segLen, overlap, acc_only):
    data, labels = datasets.load_segment_mmuisd(data_path, [0,120], segment_time=2)
    return data, labels


In [6]:
def build_model1(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(64, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [7]:
# def build_model2(input_shape, bn, reg):
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv1D, \
#                                         MaxPooling1D, \
#                                         AveragePooling1D, \
#                                         Dense, Flatten, Dropout, \
#                                         BatchNormalization, \
#                                         GlobalMaxPooling1D, \
#                                         GlobalAveragePooling1D, \
#                                         Activation
#     from tensorflow.keras import activations
#     import tensorflow.keras.regularizers as regularizers

#     model = Sequential()
#     if reg:
#         model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#         model.add(Conv1D(32, 3, strides=2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#     else:
#         model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape))
#         model.add(Conv1D(32, 3, strides=2, activation='relu'))
    
#     if bn:
#         model.add(BatchNormalization())

#     model.add(MaxPooling1D(pool_size=2))

#     if reg:
#         model.add(Conv1D(64, 5, strides=2, activation='tanh', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#         model.add(Conv1D(128, 5, strides=3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
#     else:
#         model.add(Conv1D(64, 5, strides=2, activation='tanh'))
#         model.add(Conv1D(128, 5, strides=3, activation='relu'))
    
#     if bn:
#         model.add(BatchNormalization())

#     model.add(GlobalMaxPooling1D())
#     return model

In [8]:
# def build_model3(input_shape, bn, reg):
#     from tensorflow.keras.models import Sequential
#     from tensorflow.keras.layers import Conv1D, \
#                                         MaxPooling1D, \
#                                         AveragePooling1D, \
#                                         Dense, Flatten, Dropout, \
#                                         BatchNormalization, \
#                                         GlobalMaxPooling1D, \
#                                         GlobalAveragePooling1D, \
#                                         Activation
#     from tensorflow.keras import activations
#     import tensorflow.keras.regularizers as regularizers

#     model = Sequential()
#     model.add(Conv1D(64, 5, strides=3, padding='valid', activation='relu', input_shape=input_shape))
#     if bn: model.add(BatchNormalization())
#     model.add(Conv1D(128, 3, strides=2, padding='valid', activation='relu'))
#     model.add(MaxPooling1D(pool_size=2, strides=2))
#     model.add(Conv1D(128, 2, strides=1, padding='valid', activation='tanh'))
#     model.add(MaxPooling1D(pool_size=2, strides=2))
#     model.add(GlobalMaxPooling1D())
#     return model

In [9]:
def build_model4(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(128, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(128, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [15]:
def build_model5(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(256, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(256, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [16]:
acc_only_list    = [False, True]
segLen_list      = [1, 2]
overlap_list     = [0, 0.25, 0.5, 0.75]
# models_list      = [build_model1, build_model2, build_model3]
models_list      = [build_model4, build_model5]
bn_list          = [True, False]
reg_list         = [True, False]

# Configurations
train_sps       = 500
val_sps         = 200
test_sps        = 200
max_cls         = 120
split           = [0.8, 0.1, 0.1]
output_dropout  = 0.1
runs            = 10
np.set_printoptions(precision=4)

In [17]:
def getSiameseAcc(siamese, test_set, EER_th):
    # l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]
    # print('EER threshold:', EER_th)
    # print(np.uint8(np.squeeze(siamese.predict(test_set[0])) >= EER_th))
    # print(b_labels_test)
    return np.mean(np.uint8(np.squeeze(siamese.predict(test_set[0])) >= EER_th) == b_labels_test)

def trainSVM(siamese, training_set, test_set):
    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    # classifier = SVC(kernel='poly', degree=10, C=100)
    classifier = SVC(kernel='poly', degree=5, C=10)
    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])
    
    # print(d_vect.shape)

    classifier.fit(d_vect, b_labels)
    svc_score = classifier.score(d_vect_test, b_labels_test)
    # print('SVC mean test accuracy:', svc_score*100)
    return svc_score

def trainKNN(siamese, training_set, test_set):
    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    knn = KNeighborsClassifier(n_neighbors=5)
    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])

    knn.fit(d_vect, b_labels)
    test_predictions = knn.predict(d_vect_test)
    knn_score = knn.score(d_vect_test, b_labels_test)
    # print('KNN mean test accuracy:', knn_score*100)
    return knn_score

def trainANN(siamese, training_set, test_set):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout

    l_input, r_input = training_set[0]
    b_labels = training_set[1]
    l_input_test, r_input_test = test_set[0]
    b_labels_test = test_set[1]

    feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
    d_vect = feature_exctractor.predict([l_input, r_input])
    d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])

    classifier = Sequential()
    classifier.add(Dense(8, activation='relu', input_shape=(256,)))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(8, activation='relu'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(1, activation='sigmoid'))
    classifier.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')

    classifier.fit(d_vect, b_labels, verbose=0, epochs=20, batch_size=128)
    # test_predictions = np.uint8(np.squeeze(classifier.predict(d_vect_test) >= 0.5))
    # print(test_predictions[:10])
    ann_score = classifier.evaluate(d_vect_test, b_labels_test)[1]
    # print('ANN mean test accuracy:', ann_score*100)
    return ann_score

In [18]:
def one_run(train_sps, val_sps, test_sps, max_cls, split, data, labels, lst, run_n):
    accuracies = {}
    l_input, r_input, b_labels \
        , l_input_val, r_input_val, val_labels \
            , l_input_test, r_input_test, b_labels_test = prepare_data( train_sps,
                                                                        val_sps,
                                                                        test_sps,
                                                                        max_cls,
                                                                        split,
                                                                        data,
                                                                        labels,
                                                                        [],
                                                                        run_n)
    print('Loaded data')
    adam = Adam(learning_rate=0.001)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
    
    cnn = build_model5(l_input[0].shape, bn=1, reg=0)
    siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
    siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
    print('Built model')
    if run_n == 0: 
        cnn.summary()
    print('Start training ...')
    hist = siamese.fit([l_input, r_input], 
                        b_labels, 
                        shuffle=True,
                        batch_size=64,
                        epochs=100,
                        callbacks=[earlystop],
                        validation_data=([l_input_val, r_input_val], val_labels),
                        verbose=2
                        )
    print('Finished training')
    siamese.save_weights(os.path.join(results_path, 'mmuisd', 'model_r{}_weights.h5'.format(run_n)))
    print('Saved model')
    FRR, FAR, EER, EER_th = analyze.ROC(siamese, [l_input_test, r_input_test], b_labels_test)
    print('Siamese EER:', EER)
    accuracies['siamese'] = getSiameseAcc(siamese, ([l_input_test, r_input_test], b_labels_test), EER_th)
    print('Siamese Accuracy:', accuracies['siamese'])
    accuracies['svm'] = trainSVM(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('SVM Accuracy:', accuracies['svm'])
    accuracies['ann'] = trainANN(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('ANN Accuracy:', accuracies['ann'])
    accuracies['knn'] = trainKNN(siamese, ([l_input, r_input], b_labels), ([l_input_test, r_input_test], b_labels_test))
    print('KNN Accuracy:', accuracies['knn'])


    return siamese, [FRR, FAR, EER, EER_th], accuracies

### Experiment 1: Train Different Classifiers

In [19]:
data, labels = load_dataset(2, 0.75, acc_only=False)
num_cls = 4 # number of classifiers
total_accs = np.zeros((runs, num_cls))

# Results of this configuration/experiment
# EERs = np.array([np.zeros(runs) for _ in range(3*2*2)])

for run_n in range(runs):
    siamese, ROC, accs = one_run(train_sps, val_sps, test_sps, max_cls, split, data, labels, [], run_n)
    for ii, c in enumerate(accs):
        total_accs[run_n, ii] = accs[c]
    
    np.save(os.path.join(results_path, 'mmuisd', 'classifiers.npy'), total_accs)
    
#     EERs[4 * model_index + 2*bn + reg , run_n] = EER
#     np.save(results_path + 'osaka_filters_eers2.npy', EERs)

for ii, c in enumerate(accs):
    print(total_accs[:,ii])
    print('{} accuracy: {} +/- {}'.format(c, total_accs[:,ii].mean(), total_accs[:,ii].std()))


Run #0:
Loaded data
Built model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 98, 64)            1216      
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 64)            256       
_________________________________________________________________
activation (Activation)      (None, 98, 64)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 23, 256)           82176     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 256)           1024      
_________________________________________________________________
activation_1 (Activation

Epoch 8/100
1500/1500 - 10s - loss: 0.0174 - accuracy: 0.9952 - val_loss: 0.1691 - val_accuracy: 0.9442
Epoch 9/100
1500/1500 - 9s - loss: 0.0164 - accuracy: 0.9953 - val_loss: 0.1932 - val_accuracy: 0.9519
Epoch 10/100
1500/1500 - 9s - loss: 0.0138 - accuracy: 0.9960 - val_loss: 0.2614 - val_accuracy: 0.9373
Epoch 11/100
1500/1500 - 9s - loss: 0.0146 - accuracy: 0.9957 - val_loss: 0.1892 - val_accuracy: 0.9454
Finished training
Saved model
Siamese EER: 0.04875
Siamese Accuracy: 0.9510416666666667
SVM Accuracy: 0.9529166666666666
150/150 [==============================] - 0s 1ms/step - loss: 1.9583 - acc: 0.9169
ANN Accuracy: 0.9168750047683716
KNN Accuracy: 0.9404166666666667
Run #4:
Loaded data
Built model
Start training ...
Epoch 1/100
1500/1500 - 10s - loss: 0.1501 - accuracy: 0.9523 - val_loss: 0.1710 - val_accuracy: 0.9421
Epoch 2/100
1500/1500 - 9s - loss: 0.0690 - accuracy: 0.9793 - val_loss: 0.2280 - val_accuracy: 0.9194
Epoch 3/100
1500/1500 - 9s - loss: 0.0459 - accuracy: 0.

Epoch 7/100
1500/1500 - 9s - loss: 0.0199 - accuracy: 0.9942 - val_loss: 0.2172 - val_accuracy: 0.9296
Epoch 8/100
1500/1500 - 9s - loss: 0.0192 - accuracy: 0.9941 - val_loss: 0.2179 - val_accuracy: 0.9290
Epoch 9/100
1500/1500 - 9s - loss: 0.0155 - accuracy: 0.9955 - val_loss: 0.3005 - val_accuracy: 0.9254
Epoch 10/100
1500/1500 - 9s - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.3977 - val_accuracy: 0.9152
Epoch 11/100
1500/1500 - 9s - loss: 0.0132 - accuracy: 0.9961 - val_loss: 0.3349 - val_accuracy: 0.9233
Epoch 12/100
1500/1500 - 9s - loss: 0.0125 - accuracy: 0.9966 - val_loss: 0.3974 - val_accuracy: 0.9106
Finished training
Saved model
Siamese EER: 0.04375
Siamese Accuracy: 0.9560416666666667
SVM Accuracy: 0.9141666666666667
150/150 [==============================] - 0s 1ms/step - loss: 1.0946 - acc: 0.8917
ANN Accuracy: 0.8916666507720947
KNN Accuracy: 0.9495833333333333
Run #9:
Loaded data
Built model
Start training ...
Epoch 1/100
1500/1500 - 10s - loss: 0.1518 - accuracy: 0.

In [64]:
adam = Adam(learning_rate=0.001)
cnn = build_model5(left_shape, bn=1, reg=0)
siamese = models.build_siamese(left_shape, cnn, 0.1)
siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
siamese.load_weights(os.path.join(results_path, 'osaka', 'model_r{}_weights.h5'.format(0)))

In [65]:
print('Siamese network direct test accuracy:')
siamese.evaluate([l_input_test, r_input_test], b_labels_test)

Siamese network direct test accuracy:


NameError: ignored

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM

print('Siamese network direct test accuracy:')
siamese.evaluate([l_input_test, r_input_test], b_labels_test)

print('==========SVM==========')
classifier = SVC(kernel='poly', degree=10, C=100)
feature_exctractor = Model(inputs=[siamese.get_layer('left_input').input,siamese.get_layer('right_input').input], outputs=siamese.get_layer('lambda').output)
d_vect = feature_exctractor.predict([l_input, r_input])
d_vect_test = feature_exctractor.predict([l_input_test, r_input_test])
print(d_vect.shape)

classifier.fit(d_vect, b_labels)
svc_score = classifier.score(d_vect_test, b_labels_test)
print('SVC mean test accuracy:', svc_score*100)

pca = PCA(2)
colors = ['#800000',
          '#bfef45']
plt.figure(figsize=(16,6))
plt.subplot(121)
plt.title('Ground Truth')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
plt.subplot(122)
plt.title('Prediction')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in classifier.predict(d_vect_test)], alpha=0.2)
plt.show()


# # print('==========OneClassSVM==========')

# # outlier_detector = OneClassSVM(kernel='rbf')
# # outlier_detector.fit(d_vect, b_labels)
# # test_predictions = np.uint8(outlier_detector.predict(d_vect_test) > 0)
# # svc_score = (test_predictions == b_labels_test).mean() * 100
# # print('OneClassSVM mean test accuracy:', svc_score)

# # pca = PCA(2)
# # colors = ['#800000',
# #           '#bfef45']
# # plt.figure(figsize=(16,6))
# # plt.subplot(121)
# # plt.title('Ground Truth')
# # d_vect_test_2d = pca.fit_transform(d_vect_test)
# # plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# # plt.subplot(122)
# # plt.title('Prediction')
# # d_vect_test_2d = pca.fit_transform(d_vect_test)
# # plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# # plt.show()

# print('==========ANN==========')

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout

# classifier = Sequential()
# classifier.add(Dense(8, activation='relu', input_shape=(64,)))
# classifier.add(Dropout(0.3))
# classifier.add(Dense(8, activation='relu'))
# classifier.add(Dropout(0.3))
# classifier.add(Dense(1, activation='sigmoid'))
# classifier.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')
# classifier.fit(d_vect, b_labels, verbose=0, epochs=20, batch_size=128)
# test_predictions = np.uint8(np.squeeze(classifier.predict(d_vect_test) >= 0.5))
# # print(test_predictions[:10])
# ann_score = classifier.evaluate(d_vect_test, b_labels_test)[1]
# print('ANN mean test accuracy:', ann_score*100)

# pca = PCA(2)
# colors = ['#800000',
#           '#bfef45']
# plt.figure(figsize=(16,6))
# plt.subplot(121)
# plt.title('Ground Truth')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# plt.subplot(122)
# plt.title('Prediction')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# plt.show()

# print('==========KNN==========')
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(d_vect, b_labels)
# test_predictions = knn.predict(d_vect_test)
# knn_score = knn.score(d_vect_test, b_labels_test)
# print('KNN mean test accuracy:', knn_score*100)

# pca = PCA(2)
# colors = ['#800000',
#           '#bfef45']
# plt.figure(figsize=(16,6))
# plt.subplot(121)
# plt.title('Ground Truth')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
# plt.subplot(122)
# plt.title('Prediction')
# d_vect_test_2d = pca.fit_transform(d_vect_test)
# plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
# plt.show()

print('==========GaussionMixture==========')
from sklearn.mixture import GaussianMixture

gmodel = GaussianMixture(n_components=5)
gmodel.fit(d_vect, b_labels)
test_predictions =  np.uint8((gmodel.score_samples(d_vect_test) > -55))
print(test_predictions)
gmodel_score = (test_predictions == b_labels_test).mean()
print('Gaussian Mixture mean test accuracy:', gmodel_score*100)

pca = PCA(2)
colors = ['#800000',
          '#bfef45']
plt.figure(figsize=(16,6))
plt.subplot(121)
plt.title('Ground Truth')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in b_labels_test], alpha=0.2)
plt.subplot(122)
plt.title('Prediction')
d_vect_test_2d = pca.fit_transform(d_vect_test)
plt.scatter(d_vect_test_2d[:, 0], d_vect_test_2d[:, 1], c=[colors[i] for i in test_predictions], alpha=0.2)
plt.show()